In [1]:
import pandas as pd
import pyodbc
import pgeocode
import useful_functions as use

In [2]:
username = 'vigrose'
password_edw = 'Hufflepuff~10946'
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)
password_aims = 'DQA8K@T7pXEqEt'
s = "DSN=aims_prod; UID={}; PWD={}".format(username, password_aims)
informix = pyodbc.connect(s)

In [3]:
ppd = pd.read_csv('../../Data/PPD/ppd_analysis_file.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (17,18,24,25,57,64,65,82) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
ppd['ME'] = use.fix_me(ppd.ME)
ppd['IQVIA_ME'] = [x[0:10] for x in use.fix_me(ppd.ME)]

In [4]:
#address undeliverable
#AIMS.ENTITY_COMM_EXC_CT
UNDELIVERABLE_QUERY = \
    f'''
    SELECT
    ENTITY_ID,
    COMM_ID,
    COMM_EXC_CAT_CODE
    FROM
    informix.ENTITY_COMM_EXC_CT
    WHERE
    END_DT IS NULL
    ;
    '''
undeliverable = pd.read_sql(con=informix, sql=UNDELIVERABLE_QUERY)

In [5]:
undeliverable

,entity_id,comm_id,comm_exc_cat_code
0,1241457,713984,UNDELIVER
1,1262400,720902,UNDELIVER
2,1342532,751866,UNDELIVER
3,1344430,752642,UNDELIVER
4,1346250,753452,UNDELIVER
...,...,...,...
266191,45543273,28541296,NO-AMA-SMS
266192,62977785,28541313,NO-AMA-SMS
266193,71300879,28541355,USRESEMAIL
266194,57324914,28532504,UNDELIVER


In [43]:
bad_polos = pd.merge(ppd[['TOP_CD','ENTITY_ID','POLO_COMM_ID', 'IQVIA_ME','MAILING_NAME','POLO_MAILING_LINE_1', 'POLO_MAILING_LINE_2', 'POLO_CITY', 'POLO_STATE',
       'POLO_ZIP']], undeliverable, left_on=['ENTITY_ID','POLO_COMM_ID'], right_on=['entity_id','comm_id'])

In [36]:
bad_polos.to_excel('../../Data/Undeliverable_POLOs_220509.xlsx', index=False)

In [19]:
#iqvia
username = 'vigrose'
password = 'Hufflepuff~10946'
q = "DSN=eprdods; UID={}; PWD={}".format(username, password)
ODS = pyodbc.connect(q)
iqvia_query = \
        """
        SELECT DISTINCT 
        B.PHYSICAL_ADDR_1,
        B.PHYSICAL_ADDR_2,
        B.PHYSICAL_CITY,
        B.PHYSICAL_STATE,
        B.PHYSICAL_ZIP,
        P.ME,
        A.AFFIL_IND,
        A.AFFIL_RANK
        FROM 
        ODS.ODS_IMS_BUSINESS B, ODS.SAS_ODS_IMS_PROVIDER_AFFIL A, ODS.ODS_IMS_PROFESSIONAL P
        WHERE  
        B.IMS_ORG_ID = A.IMS_ORG_ID
        AND
        A.PROFESSIONAL_ID = P.PROFESSIONAL_ID
        AND
        P.CURRENT_BATCH_FLAG='Y'
        AND
        A.CURRENT_BATCH_FLAG='Y'
        AND
        B.CURRENT_BATCH_FLAG='Y'
        """
iqvia = pd.read_sql(con=ODS, sql=iqvia_query)
iqvia.head()

,PHYSICAL_ADDR_1,PHYSICAL_ADDR_2,PHYSICAL_CITY,PHYSICAL_STATE,PHYSICAL_ZIP,ME,AFFIL_IND,AFFIL_RANK
0,201 14TH ST SW,None,LARGO,FL,33770-3133,0010200001,A,6
1,315 W HICKORY ST,None,SYLACAUGA,AL,35150-2913,0010200001,P,9
2,1613 N MCKENZIE ST,None,FOLEY,AL,36535-2247,0010200001,A,6
3,171 ASHLEY AVE,None,CHARLESTON,SC,29425-8908,0010200002,A,7
4,428 BILTMORE AVE,None,ASHEVILLE,NC,28801-4502,0010200003,A,6


In [30]:
bad_polos = bad_polos[~bad_polos.POLO_ZIP.isna()]

In [37]:
with_iqvia = pd.merge(bad_polos, iqvia, left_on='IQVIA_ME', right_on='ME')[['ME','POLO_MAILING_LINE_2','PHYSICAL_ADDR_1','AFFIL_RANK','AFFIL_IND','comm_exc_cat_code']]

In [39]:
with_iqvia.to_excel('../../Data/with_iqvia.xlsx', index=False)

In [48]:
bad_polos[bad_polos.comm_exc_cat_code=='UNDELIVER '].groupby('TOP_CD').count()[['ENTITY_ID']]

,ENTITY_ID
TOP_CD,
12,1469
20,12566
30,280
40,227
50,327
62,79
71,7402
72,363
74,79


In [47]:
bad_polos.comm_exc_cat_code.unique()

array(['UNDELIVER ', 'MAYB_UNDLV', 'NO-AMA-MAL', 'NO-CONTACT'],
      dtype=object)